**Objective**: Basic SQL Server interactions.

**Tasks**:
- Create an SQL table schema to store time-series metal prices. Include fields like Date, Metal, Price.
- Demonstrate basic CRUD operations

In this question, SQAlchemy is selected as it comes pre-installed with python3 and supports multiple database backends.

In SQLAlchemy, interactions with database can be performed with either ORM (Object-relational mapping) with a session or excecuting raw SQL queries (using `connection.execute()`). ORM is chosen for the following main reasons:
- **Object-oriented**: SQLAlchemy's ORM enables more intuitive working with Python objects as it abstracts away the underlying database structure. This enables construction of complex database queries using Pythonic syntax.
- **Security**: SQLAlchemy's ORM automatically parameterizes SQL queries, shielding the databases from attacks. In essense, it automatically handles the process of converting Python code into SQL queries and protects database in case a malicious user tries to exploit the database by injecting harmful SQL code into the queries.

# Task: Create an SQL table schema to store time-series metal prices. Include fields like Date, Metal, Price.

### Design of SQL schema

The tasks in Question 2 and 3 specify a one-table schema for the database and the following work is performed based on this requirement. Potential benefits of using schema with multiple tables is discsussed at the end of this notebook.

A one-table SQL schema is created containing price `id` (primary key), `metal`, `date`, `price`. In addion, `MACD`, `MACD_signal` and `RSI` fields are also included to prepare for requirements in Question 3. It's possible to implement future changes to the table structure; however, this may introduce complexity into the database schema, making it harder maintain.

In [1]:
# Function to print the ERD
def print_erd():
    print("+-------------+")
    print("|  MetalPrice |")
    print("+-------------+")
    print("| id (PK)     |")
    print("| metal       |")
    print("| date        |")
    print("| price       |")
    print("| MACD        |")
    print("| MACD_signal |")
    print("| RSI         |")
    print("+-------------+")

print_erd()

+-------------+
|  MetalPrice |
+-------------+
| id (PK)     |
| metal       |
| date        |
| price       |
| MACD        |
| MACD_signal |
| RSI         |
+-------------+


### Implementation

We'll import modules.

In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, ForeignKey
from sqlalchemy.orm import DeclarativeBase, sessionmaker
from datetime import date

Then we'll create `MetalPrice` table.

In [3]:
# Create Base class, for models to get SQLAlchemy ORM functionality
class Base(DeclarativeBase):
    pass

In [4]:
class MetalPrice(Base):
    __tablename__ = 'metal_prices'

    id = Column(Integer, primary_key=True)
    date = Column(Date)
    metal = Column(String)
    price = Column(Float)
    macd = Column(Float)
    macd_signal = Column(Float)
    rsi = Column(Float)

Next, we'll create an engine to manage connection to database named `metal_commodity.db`. This new database is established with `create_all()`.

In [5]:
# Create engine
engine = create_engine('sqlite:///metal_commodity.db')

# Create the table in the database
Base.metadata.create_all(engine)

# Demonstrate basic CRUD operations

We'll create a session, which represents a connection to the databse to execute queries.

In [6]:
# Create a session for interacting with the database
Session = sessionmaker(bind=engine)
session = Session()

### Create
To demonstrate create operations, we'll create 4 new records in the database. 
- Each `price` is an instance of `MetalPrice` class, representing a record in the database.
- `add_all()` will add the records to the session's transaction.
- `commit()` will save the changes made in the current session into the database.

In [7]:
# Inserting example data into MetalPrice table
price1 = MetalPrice(metal="Aluminum", date=date(2024, 4, 15), price=1800.50, macd=100, macd_signal=200, rsi=50)
price2 = MetalPrice(metal="Copper", date=date(2024, 4, 16), price=1810.75, macd=120, macd_signal=230, rsi=40)
price3 = MetalPrice(metal="Zinc", date=date(2024, 4, 15), price=29.00, macd=140, macd_signal=260, rsi=4)
price4 = MetalPrice(metal="Tin", date=date(2024, 4, 15), price=42.00, macd=61, macd_signal=90, rsi=40)

session.add_all([price1, price2, price3, price4])
session.commit()

### Read
To demonsrate read operations, we'll read all existing records in the database. 
- `query(MetalPrice)` constructs a query object for the MetalPrice table to perform the query on.
- `all()` will then return all matching records.

In [8]:
# Read from MetalPrice table
prices = session.query(MetalPrice).all()
for price in prices:
    print(f"ID: {price.id}, Metal: {price.metal}, Date: {price.date}, Price: {price.price}, MACD: {price.macd}, MACD_signal: {price.macd_signal}, RSI: {price.rsi}")

ID: 1, Metal: Aluminum, Date: 2024-04-15, Price: 1800.5, MACD: 100.0, MACD_signal: 200.0, RSI: 50.0
ID: 2, Metal: Copper, Date: 2024-04-16, Price: 1810.75, MACD: 120.0, MACD_signal: 230.0, RSI: 40.0
ID: 3, Metal: Zinc, Date: 2024-04-15, Price: 29.0, MACD: 140.0, MACD_signal: 260.0, RSI: 4.0
ID: 4, Metal: Tin, Date: 2024-04-15, Price: 42.0, MACD: 61.0, MACD_signal: 90.0, RSI: 40.0


### Update
To demonsrate update operations, we'll update `price` value of the first record (`id`=1) in the table. 
- `filter_by()` filters the query results based on the specified criteria.
- `first()` is used as we expect only one record matching the filter criteria.

In [9]:
# Updating data ID=1 in MetalPrice table from to 1820.00
price_to_update = session.query(MetalPrice).filter_by(id=1).first()
price_to_update.price = 1820.00
session.commit()

# Print updated MetalPrice table 
prices = session.query(MetalPrice).all()
for price in prices:
    print(f"ID: {price.id}, Metal: {price.metal}, Date: {price.date}, Price: {price.price}, MACD: {price.macd}, MACD_signal: {price.macd_signal}, RSI: {price.rsi}")

ID: 1, Metal: Aluminum, Date: 2024-04-15, Price: 1820.0, MACD: 100.0, MACD_signal: 200.0, RSI: 50.0
ID: 2, Metal: Copper, Date: 2024-04-16, Price: 1810.75, MACD: 120.0, MACD_signal: 230.0, RSI: 40.0
ID: 3, Metal: Zinc, Date: 2024-04-15, Price: 29.0, MACD: 140.0, MACD_signal: 260.0, RSI: 4.0
ID: 4, Metal: Tin, Date: 2024-04-15, Price: 42.0, MACD: 61.0, MACD_signal: 90.0, RSI: 40.0


### Delete
To demonsrate the delete operations, we'll delete 4th record (`id`=4) in the table.

In [10]:
# Deleting data from MetalPrice table
price_to_delete = session.query(MetalPrice).filter_by(id=4).first()
session.delete(price_to_delete)
session.commit()

# Print updated MetalPrice table 
prices = session.query(MetalPrice).all()
for price in prices:
    print(f"ID: {price.id}, Metal: {price.metal}, Date: {price.date}, Price: {price.price}, MACD: {price.macd}, MACD_signal: {price.macd_signal}, RSI: {price.rsi}")


ID: 1, Metal: Aluminum, Date: 2024-04-15, Price: 1820.0, MACD: 100.0, MACD_signal: 200.0, RSI: 50.0
ID: 2, Metal: Copper, Date: 2024-04-16, Price: 1810.75, MACD: 120.0, MACD_signal: 230.0, RSI: 40.0
ID: 3, Metal: Zinc, Date: 2024-04-15, Price: 29.0, MACD: 140.0, MACD_signal: 260.0, RSI: 4.0


To ensure Question 3 operates with a clean database, we'll clear out all existing records from the table, leaving the database empty.

In [11]:
# Delete all records from MetalPrice table
session.query(MetalPrice).delete()
session.commit()

# Print updated MetalPrice table 
prices = session.query(MetalPrice).all()
for price in prices:
    print(f"ID: {price.id}, Metal: {price.metal}, Date: {price.date}, Price: {price.price}, MACD: {price.macd}, MACD_signal: {price.macd_signal}, RSI: {price.rsi}")

Finally, we'll close the session and engine to release resources.

In [12]:
# Close session (optinal)
session.close()
engine.dispose()

# Improvement to schema

This solution was based on requirement of using one-SQL table as specified in the question briefs. Whilst a single table schema offers simplicity, a two-table schema is more suitable for larger data models.

The proposed two-table schema comprises `MetalID` and `MetalPrice` tables.
- The `MetalID` table has two columns: `id` (Primary Key) and `metal_name`.
- The `MetalPrice` table also has two columns: `id` (Primary Key) and `metal_id` (Foreign Key referencing MetalID), along with `date`, `price`, `MACD`, `MACD_signal`, `RSI columns`.
- The relationship between `MetalID` and `MetalPrice` is represented by a one-to-many (1..*) relationship. Each entry in MetalPrice is associated with one metal commodity from MetalID, while each metal commodity in MetalID can have multiple price entries in MetalPrice.

Benefits of using two-table schema:
- Flexibility: The schema allows for storing prices for various metals and accommodates potential future needs for additional fields.
- Data integrity: The foreign key relationship helps maintain consistency between metal names and their corresponding price data.
- Scalability: The design can be easily scaled to include more metals or additional data points related to the price (e.g., open, close, high, low).

In [13]:
# Function to print the ERD
def print_erd_improved():
    print("  +-------------+       +-------------+")
    print("  |   MetalID   |       |  MetalPrice |")
    print("  +-------------+       +-------------+")
    print("  | id (PK)     | 1..*  | id (PK)     |")
    print("  | metal_name  |<----->| metal_id (FK)|")
    print("  +-------------+       | date        |")
    print("                        | price       |")
    print("                        | MACD        |")
    print("                        | MACD_signal |")
    print("                        | RSI         |")
    print("                        +-------------+")

# Print the ERD
print_erd_improved()


  +-------------+       +-------------+
  |   MetalID   |       |  MetalPrice |
  +-------------+       +-------------+
  | id (PK)     | 1..*  | id (PK)     |
  | metal_name  |<----->| metal_id (FK)|
  +-------------+       | date        |
                        | price       |
                        | MACD        |
                        | MACD_signal |
                        | RSI         |
                        +-------------+


This two-table schema can be created using the following classes.

In [ ]:
# Define MetalID table
class MetalID(Base):
    __tablename__ = 'metal'

    id = Column(Integer, primary_key=True)
    metal_name = Column(String)
    prices = relationship('MetalPrice', backref='metal') # Relate MetalPrice instances to MetalID via attribute 'metal'

# Define MetalPrice table with foreign key relationship
class MetalPrice(Base):
    __tablename__ = 'metal_price'

    id = Column(Integer, primary_key=True)
    metal_id = Column(Integer, ForeignKey('metal.id'))
    date = Column(Date)
    price = Column(Float)
    macd = Column(Float)
    macd_signal = Column(Float)
    rsi = Column(Float)    